**<h2><center>Análise preditiva de clientes potenciais com random forest<center><h2>**

**Importando bibliotecas necessárias**

In [237]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

**importando a base**


In [238]:
df = pd.read_excel('Forms_ConstruConecta.xlsx')
display(df)

,Índice,Dificuldade_Materiais_Pessoas,Faixa_Etaria,Realizou_Obra,Desafios,Satisfacao_Atual,Realizaria_Reforma,Condicoes_Financeiras,Usuaria_App,Valorizacao_Sustentabilidade,Funcionalidade_no_App
0,0,Não,18 - 20,Não,Não Informado,Sim,Sim,Não,Sim,Importante,Não Informado
1,1,Sim,18 - 20,Sim,Custo,Não,Sim,Não,Sim,Muito Importante,Contratação de serviços
2,2,Sim,31 - 50,Sim,Profissionais Qualificados,Não,Sim,Sim,Sim,Muito Importante,"Disponibilidade, prazo, custo, qualidade"
3,3,Sim,51 - 70,Não,Não Informado,Não,Sim,Sim,Sim,Muito Importante,Localização das iportunidades
4,4,Sim,71 - 80,Sim,Profissionais Qualificados,Sim,Sim,Não,Sim,Importante,Seviços de profiossionais e materiais de const...
...,...,...,...,...,...,...,...,...,...,...,...
267,267,Sim,51 - 70,Sim,Profissionais Qualificados,Sim,Sim,Não,Sim,Importante,Bastante relevante
268,268,Sim,51 - 70,Não,Não Informado,Não,Sim,Não Informado,Sim,Muito Importante,Economia e sustentabilidade
269,269,Sim,31 - 50,Sim,Profissionais Qualificados,Não,Sim,Não,Não,Importante,Não sei
270,270,Sim,31 - 50,Sim,Profissionais Qualificados,Não,Sim,Não,Sim,Muito Importante,Seriedade e acessibilidade


**Tirando as colunas Faixa_Etária e Funcionalidade_no_App**

In [239]:
df = df.drop(['Faixa_Etaria', 'Funcionalidade_no_App', 'Índice'], axis=1)
#tratando erros
df['Realizaria_Reforma'] = df['Realizaria_Reforma'].replace('Sim;Não', 'Sim')
display(df)


,Dificuldade_Materiais_Pessoas,Realizou_Obra,Desafios,Satisfacao_Atual,Realizaria_Reforma,Condicoes_Financeiras,Usuaria_App,Valorizacao_Sustentabilidade
0,Não,Não,Não Informado,Sim,Sim,Não,Sim,Importante
1,Sim,Sim,Custo,Não,Sim,Não,Sim,Muito Importante
2,Sim,Sim,Profissionais Qualificados,Não,Sim,Sim,Sim,Muito Importante
3,Sim,Não,Não Informado,Não,Sim,Sim,Sim,Muito Importante
4,Sim,Sim,Profissionais Qualificados,Sim,Sim,Não,Sim,Importante
...,...,...,...,...,...,...,...,...
267,Sim,Sim,Profissionais Qualificados,Sim,Sim,Não,Sim,Importante
268,Sim,Não,Não Informado,Não,Sim,Não Informado,Sim,Muito Importante
269,Sim,Sim,Profissionais Qualificados,Não,Sim,Não,Não,Importante
270,Sim,Sim,Profissionais Qualificados,Não,Sim,Não,Sim,Muito Importante


**Separação de atributos e resposta**

In [240]:
df_X = df.drop(columns=['Usuaria_App'])
df_y = df['Usuaria_App']

print("Atributos:\n", df_X.head())
print("\nResposta:\n", df_y.head())

Atributos:
   Dificuldade_Materiais_Pessoas Realizou_Obra                    Desafios  \
0                           Não           Não               Não Informado   
1                           Sim           Sim                       Custo   
2                           Sim           Sim  Profissionais Qualificados   
3                           Sim           Não               Não Informado   
4                           Sim           Sim  Profissionais Qualificados   

  Satisfacao_Atual Realizaria_Reforma Condicoes_Financeiras  \
0              Sim                Sim                   Não   
1              Não                Sim                   Não   
2              Não                Sim                   Sim   
3              Não                Sim                   Sim   
4              Sim                Sim                   Não   

  Valorizacao_Sustentabilidade  
0                   Importante  
1             Muito Importante  
2             Muito Importante  
3             

**Transformando dados qualitativos em quantitativos**

In [241]:

le = LabelEncoder()

colunas=df_X.columns

for col in colunas:
    df_X[col] = le.fit_transform(df_X[col])

df_y=le.fit_transform(df_y)

display("Atributos:", df_X)
print("Resposta:\n", df_y)

# Converter em valores numpy para treino
atributo_treino_df = df_X.values
resposta_treino_df = df_y


'Atributos:'

,Dificuldade_Materiais_Pessoas,Realizou_Obra,Desafios,Satisfacao_Atual,Realizaria_Reforma,Condicoes_Financeiras,Valorizacao_Sustentabilidade
0,0,0,3,1,1,0,0
1,1,1,1,0,1,0,1
2,1,1,4,0,1,2,1
3,1,0,3,0,1,2,1
4,1,1,4,1,1,0,0
...,...,...,...,...,...,...,...
267,1,1,4,1,1,0,0
268,1,0,3,0,1,1,1
269,1,1,4,0,1,0,0
270,1,1,4,0,1,0,1


Resposta:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1
 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1]


**Separando os dados entre conjuntos de treino e teste**

In [242]:
X_treino, X_teste, y_treino, y_teste = train_test_split(df_X, df_y, test_size=0.3, random_state=42)

print("Conjunto de Treino:")
display(X_treino)
display(y_treino)

print("\nConjunto de Teste:")
display(X_teste)
display(y_teste)

Conjunto de Treino:


,Dificuldade_Materiais_Pessoas,Realizou_Obra,Desafios,Satisfacao_Atual,Realizaria_Reforma,Condicoes_Financeiras,Valorizacao_Sustentabilidade
251,1,1,1,0,1,2,1
92,0,1,4,1,1,2,1
201,0,1,4,0,1,0,1
126,1,1,4,1,1,2,0
124,1,1,4,0,1,2,1
...,...,...,...,...,...,...,...
188,1,1,3,1,0,0,2
71,1,1,4,0,1,0,1
106,0,1,1,1,1,0,1
270,1,1,4,0,1,0,1


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])


Conjunto de Teste:


,Dificuldade_Materiais_Pessoas,Realizou_Obra,Desafios,Satisfacao_Atual,Realizaria_Reforma,Condicoes_Financeiras,Valorizacao_Sustentabilidade
30,1,0,3,1,0,0,0
116,1,0,3,1,0,2,0
79,1,0,2,0,1,2,1
127,1,1,4,1,1,2,1
196,1,1,2,1,1,0,1
...,...,...,...,...,...,...,...
97,1,1,4,1,0,2,0
200,1,1,0,1,1,2,1
38,1,1,0,0,1,2,1
154,0,1,3,1,0,2,0


array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

**Utilizando o modelo Random Forest Classifier**

In [243]:
# MELHOR MODELO
#o modelo base que está sendo ajustado é um Random Forest Classifier. O GridSearchCV está varrendo diferentes combinações
#de hiperparâmetros para este modelo, ajustando o número de árvores (n_estimators) e a profundidade máxima de cada árvore (max_depth).
#parametros p serem testados

#hiper parametros utilizados na implentação da Random Forest
#criterion (critério de divisão das árvores) -não
#n_estimators: O número de árvores de decisão no floresta -usar
#min_samples_split (número mínimo de amostras para dividir um nó) -não
#max_depth: A profundidade máxima de cada árvore -usar

param_grid = {
    'n_estimators': [100],
    'max_depth': [None, 5, 10]
}

rf = RandomForestClassifier()
# fazendo busca no espaço de parâmetros, e testando todas as combinações possíveis de valores para os hiperparâmetros q foram colocados em param_grid.
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_treino, y_treino)

# validação cruzada feita com 5 folds. conjunto de treinamento está sendo dividido em 5 partes iguais,
#ent cada iteração, um dos folds está sendo usado como conjunto de teste, enquanto os outros 4 são usados p treinar o modelo.

# Melhor modelo
#dps de testar todas as combinações, o GridSearchCV pega o modelo que teve a melhor performance média na validação cruzada e armazena o modelo em best_model.
best_model = grid_search.best_estimator_
#print(best_model)

print(best_model)
print(best_model.get_params())
# OBS: os valores de n_estimators e max_depth no grid de parametros q estão no best_model maximizaram o desempenho do modelo na validação cruzada c 5 folds

RandomForestClassifier(max_depth=5)
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


**Testando o modelo**

In [244]:
#TESTANDO MELHOR MODELO
y_pred = best_model.predict(X_teste)


**Avaliando o modelo**

In [245]:
#avaliando modelo
print(classification_report(y_teste, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.93      0.97      0.95        76

    accuracy                           0.90        82
   macro avg       0.46      0.49      0.47        82
weighted avg       0.86      0.90      0.88        82



In [246]:
#acurácia
acuracia = accuracy_score(y_teste, y_pred)
print("Acurácia:", acuracia)

Acurácia: 0.9024390243902439


**Testando o modelo com dados da base**

In [247]:
teste1=[0,0,3,1,1,0,0]
teste2=[1,1,1,0,1,0,1]
teste3=[1,1,4,0,1,2,1]
teste4=[1,0,3,0,1,2,1]

atributos=[teste1,teste2,teste3,teste4]
resposta=[1,1,1,1]
#atributos=[teste1]
#resposta=[1]

#df_x_teste=pd.DataFrame(atributos)
#df_y_teste=pd.DataFrame(resposta)
#display(df_x_teste)

#display(df_y_teste)

In [248]:
#classificacao=classificador_bayes.predict(atributos)
classificacao=best_model.predict(atributos)
print("resposta Certa :)")
print(resposta)
print("classificacao da IA: ")
print(classificacao)

resposta Certa :)
[1, 1, 1, 1]
classificacao da IA: 
[1 1 1 1]


C:\Users\talitanalon-ieg\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
